# Pre-processing (Complete)

Data Preprocessing: https://www.kaggle.com/khairulislam/data-preprocessing/notebook

## imports

In [1]:
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# metrics are used to find accuracy or error
from sklearn import metrics

C:\Users\Sajepan\.conda\envs\MasterThesisGPU\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Sajepan\.conda\envs\MasterThesisGPU\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Sajepan\.conda\envs\MasterThesisGPU\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Functions

In [2]:
# 1. Reading Train and test dataset.
# 2. Check if dataset is reversed.
# 3. Drop 'id', and 'attack_cat' columns.
def import_train_test():
    train = pd.read_csv('../Dataset/UNSW_NB15_training-set.csv')
    test = pd.read_csv('../Dataset/UNSW_NB15_testing-set.csv')
    
    # Dropping the columns based on Feature Selection:
    # https://www.kaggle.com/khairulislam/unsw-nb15-feature-importance
    drop_cols = ['attack_cat', 'id'] + ['response_body_len', 'spkts', 'ct_flw_http_mthd', 'trans_depth', 'dwin', 'ct_ftp_cmd', 'is_ftp_login']
    for df in [train, test]:
        for col in drop_cols:
            if col in df.columns:
                print('Dropping: ', col)
                df.drop([col], axis=1, inplace=True)
    
    if train.shape < test.shape:
        # Reversing the dataset
        train, test = test, train
        print("Train and Test sets are reversed, Corrected Shape:")
        print("Train shape: ", train.shape)
        print("Test shape: ", test.shape)
    else:
        print("The dataset, is already reversed")
        print("Train shape: ", train.shape)
        print("Test shape: ", test.shape)
    return train, test

In [3]:
def feature_engineer(df):
    # Everything except: 'FIN', 'INT', 'CON', 'REQ', 'RST is renamed 'others'
    df.loc[~df['state'].isin(['FIN', 'INT', 'CON', 'REQ', 'RST']), 'state'] = 'others'
    # Everything except: ''-', 'dns', 'http', 'smtp', 'ftp-data', 'ftp', 'ssh', 'pop3' is renamed 'others'
    df.loc[~df['service'].isin(['-', 'dns', 'http', 'smtp', 'ftp-data', 'ftp', 'ssh', 'pop3']), 'service'] = 'others'
    # Merging 'igmp', 'icmp', 'rtp' into one protocol: 'igmp_icmp_rtp'
    df.loc[df['proto'].isin(['igmp', 'icmp', 'rtp']), 'proto'] = 'igmp_icmp_rtp'
    # Everything except: 'tcp', 'udp' ,'arp', 'ospf', 'igmp_icmp_rtp' is renamed to 'others'
    df.loc[~df['proto'].isin(['tcp', 'udp','arp', 'ospf', 'igmp_icmp_rtp']), 'proto'] = 'others'
    return df

In [4]:
def get_cat_columns(train):
    # Defining an empty list
    categorical = []
    # Iterating through the columns and checking for columns with datatyp "Object"
    for col in train.columns:
        if train[col].dtype == 'object':
            categorical.append(col) # appending "object" columns to categorical
    return categorical

## Pre-processing

In [5]:
# Importing train test by using the function
train, test = import_train_test()

Dropping:  attack_cat
Dropping:  id
Dropping:  response_body_len
Dropping:  spkts
Dropping:  ct_flw_http_mthd
Dropping:  trans_depth
Dropping:  dwin
Dropping:  ct_ftp_cmd
Dropping:  is_ftp_login
Dropping:  attack_cat
Dropping:  id
Dropping:  response_body_len
Dropping:  spkts
Dropping:  ct_flw_http_mthd
Dropping:  trans_depth
Dropping:  dwin
Dropping:  ct_ftp_cmd
Dropping:  is_ftp_login
Train and Test sets are reversed, Corrected Shape:
Train shape:  (175341, 36)
Test shape:  (82332, 36)


In [6]:
# To check if train and test datasets inhibits missing values
train.isnull().sum()
test.isnull().sum()

dur                 0
proto               0
service             0
state               0
dpkts               0
sbytes              0
dbytes              0
rate                0
sttl                0
dttl                0
sload               0
dload               0
sloss               0
dloss               0
sinpkt              0
dinpkt              0
sjit                0
djit                0
swin                0
stcpb               0
dtcpb               0
tcprtt              0
synack              0
ackdat              0
smean               0
dmean               0
ct_srv_src          0
ct_state_ttl        0
ct_dst_ltm          0
ct_src_dport_ltm    0
ct_dst_sport_ltm    0
ct_dst_src_ltm      0
ct_src_ltm          0
ct_srv_dst          0
is_sm_ips_ports     0
label               0
dtype: int64

In [7]:
# Addressing the different Data types for each column
train.dtypes
test.dtypes

dur                 float64
proto                object
service              object
state                object
dpkts                 int64
sbytes                int64
dbytes                int64
rate                float64
sttl                  int64
dttl                  int64
sload               float64
dload               float64
sloss                 int64
dloss                 int64
sinpkt              float64
dinpkt              float64
sjit                float64
djit                float64
swin                  int64
stcpb                 int64
dtcpb                 int64
tcprtt              float64
synack              float64
ackdat              float64
smean                 int64
dmean                 int64
ct_srv_src            int64
ct_state_ttl          int64
ct_dst_ltm            int64
ct_src_dport_ltm      int64
ct_dst_sport_ltm      int64
ct_dst_src_ltm        int64
ct_src_ltm            int64
ct_srv_dst            int64
is_sm_ips_ports       int64
label               

In [ ]:
# Splitting the dataset into inputs and outputs
x_train, y_train = train.drop(['label'], axis=1), train['label']
x_test, y_test = test.drop(['label'], axis=1), test['label']
# Running the inputs into the feature_engineer function
x_train, x_test = feature_engineer(x_train), feature_engineer(x_test)

In [ ]:
# Getting the categorical and non categorical columns
categorical_columns = get_cat_columns(x_train)
non_categorical_columns = [x for x in x_train.columns if x not in categorical_columns]

In [ ]:
x_train.head()

In [ ]:
# Using standard scaler to normalize data on non categorical columns
scaler = StandardScaler()
x_train[non_categorical_columns] = scaler.fit_transform(x_train[non_categorical_columns])
x_test[non_categorical_columns] = scaler.transform(x_test[non_categorical_columns])

In [ ]:
x_train

In [ ]:
# Using get_dummies to make the categorical values usable.
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
print("Column mismatch {0}, {1}".format(set(x_train.columns)- set(x_test.columns),  set(x_test.columns)- set(x_train.columns)))
features = list(set(x_train.columns) & set(x_test.columns))

In [ ]:
features = list(set(x_train.columns) & set(x_test.columns))

In [ ]:
print(f"Number of features {len(features)}")
x_train = x_train[features]
x_test = x_test[features]

In [ ]:
x_train

In [ ]:
print('X_train Shape: ', x_train.shape)
print('y_train Shape: ', y_train.shape)
print('X_test Shape: ', x_test.shape)
print('y_test Shape: ', y_test.shape)

## Export CSV

In [ ]:
# merge x_train and y_train before exporting to CSV
x_train['label'] = y_train
x_test['label'] = y_test
x_train.to_csv('../Dataset/train_pp4.csv', index=False)
x_test.to_csv('../Dataset/test_pp4.csv', index=False)